# Schleswig-Holstein

Every federal state is represented by its own input directory and is processed into a NUTS level 2 directory containing a sub-folder for each discharge location. These folder names are derived from NUTS and reflect the CAMELS id. The NUTS level 2 code for Schleswig Holstein is `DEF`.

To pre-process the data, you need to write (at least) two functions. One should extract all metadata and condense it into a single `pandas.DataFrame`. This is used to build the folder structure and derive the ids.
The second function has to take an id, as provided by the state authorities, called `provider_id` and return a `pandas.DataFrame` with the transformed data. The dataframe needs the three columns `['date', 'q' | 'w', 'flag']`.

For easier and unified output handling, the `camelsp` package contains a context object called `Bundesland`. It takes a number of names and abbreviations to identify the correct federal state and returns an object that holds helper and save functions.

The context saves files as needed and can easily be changed to save files with different strategies, ie. fill missing data with NaN, merge data into a single file, create files for each variable or pack everything together into a netcdf.

In [ ]:
import pandas as pd
import numpy as np
from pandas.errors import ParserError
import os
from glob import glob
from pprint import pprint
from tqdm import tqdm
from typing import Union, Dict, Tuple
from datetime import datetime as dt
from dateparser import parse
import warnings
from io import BytesIO

from camelsp import Bundesland

The context can also be instantiated as any regular Python class, ie. to load only the default input data path, that we will user later.

In [ ]:
# the context also makes the input path available, if camelsp was install locally
BASE = Bundesland('Schleswig-Holstein').input_path
BASE

## Parse metadata

Pegel metadata can be read quite easily. Only the separator is important as we have whitespaces in `'Gewässer'`

In [ ]:
with Bundesland('Schleswig-Holstein') as bl:
    metadata = pd.read_csv(os.path.join(bl.input_path, 'gauge_attributes.csv'), encoding='latin1')

metadata

In [ ]:
# The id_column is id.
id_column = 'id'

## Load Data

Loading data is a bit more complicated. There is an extra header, but that is not important. Only the 'Einhait' contains important information, but that has been added manually to the metadata.

In [ ]:
# we only have Q, so remove variable from args
def extract_file(nr: Union[str, int], base_path: str) -> Tuple[Dict[str, str], pd.DataFrame]:
    # always use str ids
    nr = str(nr)
    # merge the nr
    if '.' in nr:
        nr = nr.replace('.', '')

    # build the path
    path = os.path.join(base_path, 'Q' , f'{nr}.csv')

    # return empty dataframe if data does not exist
    if not os.path.exists(path):
        return pd.DataFrame(columns=['date', 'q', 'flag'])
    
    # otherwise read
    df = pd.read_csv(path, encoding="latin1", sep=";", decimal=",", parse_dates=[0], dayfirst=True)
    
    df.columns = ['date', 'q', 'flag']
    df['q'] = df.q.astype(float)

    # check if there are comments
    if df.q.isna().all():
        return pd.DataFrame(columns=['date', 'q', 'flag'])
    
    # build the flag column
    df['flag'] = df.flag.apply(lambda f: f.lower() == 'qualitätsgesichert')

    return df

extract_file(114614, BASE)

### Finally run

Now, the Q and W data can be extracted. The cool thing is, that all the id creation, data creation, merging and the mapping from our ids to the original ids and files is done by the context. This is helpful, as we less likely screw something up.

In [ ]:
with Bundesland('Schleswig-Holstein') as bl:
    # save the metadata
    bl.save_raw_metadata(metadata, id_column, overwrite=True)

    # for reference, call the nuts-mapping as table
    nuts_map = bl.nuts_table
    print(nuts_map.head())

    # go for each    
    for _id in tqdm(metadata[id_column].values):
        provider_id = str(_id)
        # extract the two files
        q_df = extract_file(provider_id, bl.input_path)
        
        # always empty
        w_df = pd.DataFrame(columns=['date', 'w', 'flag'])

        # save
        bl.save_timeseries(q_df, provider_id)
        bl.save_timeseries(w_df, provider_id)
